In [1]:
from datetime import datetime

import pytz
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

local_tz = pytz.timezone("Asia/Dubai")

In [2]:
date_fr = (
    datetime.now().replace(hour=0, minute=0, second=0).strftime("%m/%d/%Y %H:%M:%S")
)
date_to = datetime.now(local_tz).strftime("%m/%d/%Y %H:%M:%S")

In [3]:
db = Database()

get_hotel_mapping = db.get_hotel_mapping_as_df()
get_room_mapping = db.get_room_mapping_as_df()
get_meal_mapping = db.get_meal_mapping_as_df()
get_operator_mapping = db.get_operator_mapping_as_df()
reservation = db.reservations_booking

hotel_mapping = dict(
    zip(get_hotel_mapping["external_code"], get_hotel_mapping["hotel_id"])
)
room_mapping = dict(
    zip(
        get_room_mapping.apply(lambda x: (x["external_code"], x["hotel_id"]), axis=1),
        get_room_mapping["id"],
    )
)
meal_mapping = dict(zip(get_meal_mapping["code"], get_meal_mapping["id"]))

operator_mapping = dict(
    zip(get_operator_mapping["external_code"], get_operator_mapping["operator_id"])
)

/home/jovyan/work/database.py:12: SAWarning: Did not recognize type 'geometry' of column 'geom'
  self.mapping_hotel = Table(


In [4]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        #(
        #    "data_encoder",
        #    BookingDataEncoder(
        #        hotel_mapping,
        #        room_mapping,
        #        meal_mapping,
        #        operator_mapping,
        #    ),
        #),
    ]
)

In [5]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 3)
    get_bookings = booking_data.get()

    df = pipeline.fit_transform(get_bookings)

    if df is not None:
        dfs.append(df)
        #stmt = insert(reservation).values(df.to_dict(orient="records"))
        #stmt = stmt.on_conflict_do_update(
        #    constraint="reservations_booking_pkey", set_=dict(stmt.excluded)
        #)

        #with db.engine.begin() as connection:
        #    connection.execute(stmt)

In [7]:
import pandas as pd

bookings = pd.concat(dfs, axis=0, ignore_index=True)
bookings.to_csv("bookings.csv", index=False)